In [2]:
import requests
from bs4 import BeautifulSoup


In [3]:
# Page content from Website URL 
page = requests.get('https://www.indyskipass.com/our-resorts') 


In [24]:
import re

def add_space_between_letters_and_numbers(text):
    # Use regex to add space between letters and numbers
    updated_text = re.sub(r'(?<=[a-zA-Z])(?=\d)|(?<=\d)(?=[a-zA-Z])', ' ', text)
    return updated_text




abc 123 xyz 456


In [53]:
from shapely.geometry import Point
# parse html content 
soup = BeautifulSoup( page.content , 'html.parser') 
# Selector to find all <a> elements with specific patterns
#resorts = soup.select('a.node--type-resort')
parent_div = soup.find('div',class_ = 'content isoGrid')
resorts = parent_div.find_all("a",recursive=False)
resort_list = []
for resort in resorts:
    location = resort.get("data-location")
    # Remove 'POINT(' and the trailing ')'
    if location != None:
        coordinates = location.replace("POINT (", "").replace(")", "").strip()
        # Split the string into longitude and latitude
        longitude, latitude = map(float, coordinates.split())
        isnordic = resort.get("data-isnordic")
        isxconly = resort.get('data-isxconly')
        isalpinexc = resort.get('data-isalpinexc')
        isallied = resort.get('data-isallied')
        resortname = resort.find('span', class_='label').text
        resort_ul = resort.find('ul',class_='d-flex reset jc-space-between card-resort__list cardList')
        resort_li = resort_ul.find_all('li')
        url = 'https://www.indyskipass.com'+ resort.get('href')
        
        
        resort_obj = {
            "resortname": resortname,
            "isnordic": isnordic.title(),
            "isxconly": isxconly.title(),
            "isalpinexc": isalpinexc.title(),
            "isallied": isallied.title(),
            "url": url,
            "geometry": Point(longitude, latitude)
        }
        
        
        
        for li in resort_li:
            label = li.find(class_='d-block label lh-1')
            #print(label)
            value = li.find(class_='value lh-1')
            resort_obj[label.get_text()] = value.get_text().strip()

        
        
        resort_list.append(resort_obj)
for resort in resort_list:
    resort['Vertical'] = add_space_between_letters_and_numbers(resort['Vertical'])
    if resort['Lifts'] == '- -':
        resort['Lifts'] = 0
    if resort['Vertical'] == '- -':
        resort['Vertical'] = 0
    if resort['Trails'] == '- -':
        resort['Trails'] = 0


In [54]:
resort_list

[{'resortname': '49 Degrees North',
  'isnordic': 'True',
  'isxconly': 'False',
  'isalpinexc': 'True',
  'isallied': 'False',
  'url': 'https://www.indyskipass.com/our-resorts/49-degrees-north',
  'geometry': <POINT (-118 48.3)>,
  'Vertical': '1851 ft',
  'Trails': '88',
  'Lifts': '7',
  'Night': 'No',
  'T-Parks': 'Yes'},
 {'resortname': 'Amihari Onsen Ski Resort',
  'isnordic': 'False',
  'isxconly': 'False',
  'isalpinexc': 'False',
  'isallied': 'False',
  'url': 'https://www.indyskipass.com/our-resorts/amihari-onsen-ski-resort',
  'geometry': <POINT (141 39.8)>,
  'Vertical': '1969 ft',
  'Trails': '7',
  'Lifts': '4',
  'Night': 'No',
  'T-Parks': 'No'},
 {'resortname': 'Andes Tower Hills',
  'isnordic': 'True',
  'isxconly': 'False',
  'isalpinexc': 'True',
  'isallied': 'False',
  'url': 'https://www.indyskipass.com/our-resorts/andes-tower-hills',
  'geometry': <POINT (-95.6 45.9)>,
  'Vertical': '290 ft',
  'Trails': '16',
  'Lifts': '6',
  'Night': 'Yes',
  'T-Parks': 'Ye

In [55]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(resort_list, crs="EPSG:4326")
gdf.to_file('./indypass.geojson', driver="GeoJSON")
